<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/SudokuSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc -q https://github.com/ollihansen90/Mathe-SH/raw/main/utils/utils_sudoku.py

In [8]:
from utils_sudoku import erstelle_sudoku, plotte
from copy import deepcopy

In [7]:
def initialisiere():
    eintraege = "123456789"
    liste = []
    for i in range(9):
        zeile = []
        for j in range(9):
            zeile.append(eintraege)
        liste.append(zeile)
    return liste

def einlesen(solver, sudoku_leer):
    s = deepcopy(solver)
    for i in range(9):
        for j in range(9):
            if sudoku_leer[i][j]!=" ":
                s[i][j] = sudoku_leer[i][j]
    return s

def plotte_solver(solver):
    s = deepcopy(solver)
    output = []
    for i in range(9):
        zeile = ""
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)!=1:
                zeile = zeile+" "
            else:
                zeile = zeile+eintrag
        output.append(zeile)
    plotte(output)

def loesche_doppelte(solver):
    s = deepcopy(solver)
    for i in range(9):
        zeile = s[i]
        bekannte = ""
        for j in range(9):
            eintrag = zeile[j]
            if len(eintrag)==1:
                bekannte = bekannte+eintrag

        # Schritt 1: Finde Eintrag mit len(eintrag)>1
        for k in range(9):
            eintrag = zeile[k]
            if len(eintrag) != 1:

                # Schritt 2: Eliminiere alle "bekannte" aus "eintrag"
                for l in bekannte:
                    eintrag = eintrag.replace(str(l), "")

            # Schritt 3: Überschreibe Zeile s[i]
            s[i][k] = eintrag
    return s

def transponiere_solver(solver):
    s = deepcopy(solver)
    out = []
    for i in range(9):
        inner = []
        for j in range(9):
            inner.append(s[j][i])
        out.append(inner)
    return out

def loesche_spaltenweise(solver):
    s = deepcopy(solver)
    s = transponiere_solver(s)
    s = loesche_doppelte(s)
    s = transponiere_solver(s)
    return s

def loesche_3x3(solver):
    s = deepcopy(solver)
    bekannt = getblock(s)
    for i in range(9):
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)>1:
                block_i = i//3
                block_j = j//3
                for zahl in bekannt[block_i][block_j]:
                    eintrag = eintrag.replace(zahl, "")
            s[i][j] = eintrag
    return s

def getblock(solver):
    s = deepcopy(solver)
    out = [["","",""],["","",""],["","",""]]
    for i in range(9):
        for j in range(9):
            eintrag = s[i][j]
            if len(eintrag)==1:
                block_i = i//3
                block_j = j//3
                out[block_i][block_j] = out[block_i][block_j]+eintrag
    return out

def loese(solver):
    s = deepcopy(solver)
    s_alt = None
    while s!=s_alt:
        s_alt = deepcopy(s)
        s = loesche_doppelte(s)
        s = loesche_spaltenweise(s)
        s = loesche_3x3(s)
    return s

def beseitige_zp(solver):
    s = deepcopy(solver)
    for i in range(9):
        zweier = []
        for j in range(9):
            if len(s[i][j])==2:
                zweier.append(s[i][j])
        if len(zweier)>1:
            loeschlist = []
            for h in range(0, len(zweier)-1):
                for k in range(h+1, len(zweier)):
                    if zweier[h]==zweier[k]:
                        loeschlist.append(zweier[h])
            if len(loeschlist)>0:
                for l in loeschlist:
                    for j in range(9):
                        if len(s[i][j])>2:
                            s[i][j] = s[i][j].replace(l[0], "")
                            s[i][j] = s[i][j].replace(l[1], "")
    return s

def beseitige_zp_spalte(solver):
    s = deepcopy(solver)
    s = transponiere_solver(s)
    s = beseitige_zp(s)
    return transponiere_solver(s)

def loese(solver):
    s = deepcopy(solver)
    s_alt = None
    while s!=s_alt:
        s_alt = deepcopy(s)
        s = loesche_doppelte(s)
        s = loesche_spaltenweise(s)
        s = loesche_3x3(s)
        s = beseitige_zp(s)
        s = beseitige_zp_spalte(s)
    return s